# Imports

In [2]:
import pandas as pd
import openpyxl 

# Data

In [3]:
dir_path = "C:/Users/CPL17/OneDrive/Desktop/"
full_path = dir_path + "PA Wildflower Database.xlsx"

In [4]:
local_long = pd.read_excel(full_path,sheet_name="LOCAL")

In [5]:
metadata= pd.read_excel(full_path,sheet_name="ERA_PA")

In [6]:
local_long.shape

(2181, 6)

In [7]:
local_long = local_long.iloc[:,0:3]

In [8]:
local_long.head()

,LOCAL,SOURCE,SOURCE_URLS
0,AGAL5,Arcadia Washington,https://arcadianatives.com/
1,AGFO,Arcadia Washington,https://arcadianatives.com/
2,AGSC,Arcadia Washington,https://arcadianatives.com/
3,AGTE3,Arcadia Washington,https://arcadianatives.com/
4,ALCE2,Arcadia Washington,https://arcadianatives.com/


In [9]:
local_long["SOURCE"].value_counts()

Gino's Newtown                            295
Redbud Media                              278
Edge of the Woods Orefield                168
Collins Nursery Glenside                  164
Arcadia Washington                        158
Archewild Quakertown                      158
Hungry Hook Bainbridge                    152
Northeast Native Perennials Quakertown    142
Perennial New Bloomfield                  138
Keystone Robesonia                        127
Go Native Trees                           125
Bowman's New Hope                          82
Musser Indiana                             65
Butterfly Oasis Lancaster                  38
Earthbound Ephrata                         33
Good Host Plants Philly                    30
Northbrook Natives West Chester            28
Name: SOURCE, dtype: int64

In [10]:
local_long["LOCAL"].value_counts().iloc[[0,-1]]

MOFI     14
HIKA2     1
Name: LOCAL, dtype: int64

# Aggregations

In [11]:
f = lambda x: ', '.join(map(str, set(x)))
local_agg = local_long.groupby("LOCAL").agg({"SOURCE_URLS":[f,len]})

In [12]:
local_agg.reset_index(inplace=True)
local_agg.columns = ["USDA Symbol","URL","COUNT"]
local_agg.head()

,USDA Symbol,URL,COUNT
0,ABBA,"https://www.musserforests.com/, http://www.per...",4
1,ACAM,https://edgeofthewoodsnursery.com/,1
2,ACMI2,https://www.ginosnursery.com/,1
3,ACNE2,"https://archewild.com/, https://www.gonativetr...",3
4,ACPA,"https://www.collinsnursery.com/, https://www.n...",5


In [13]:
(local_agg["USDA Symbol"].value_counts() != 1).sum() #Ensure Properly Aggregate

0

# Join with MetaData

In [14]:
metadata = metadata[["USDA Symbol","Scientific Name","Common Name"]]

In [15]:
metadata.head(3)

,USDA Symbol,Scientific Name,Common Name
0,ABBA,Abies balsamea,Balsam Fir
1,ACNE2,Acer negundo,Boxelder
2,ACPE,Acer pensylvanicum,Striped Maple


In [16]:
set(local_agg["USDA Symbol"]).difference(set(metadata["USDA Symbol"]))

set()

In [17]:
final = local_agg.merge(metadata,on="USDA Symbol",how="left")
final.head()

,USDA Symbol,URL,COUNT,Scientific Name,Common Name
0,ABBA,"https://www.musserforests.com/, http://www.per...",4,Abies balsamea,Balsam Fir
1,ACAM,https://edgeofthewoodsnursery.com/,1,Acorus americanus,Sweetflag
2,ACMI2,https://www.ginosnursery.com/,1,Achillea millefolium,Common Yarrow
3,ACNE2,"https://archewild.com/, https://www.gonativetr...",3,Acer negundo,Boxelder
4,ACPA,"https://www.collinsnursery.com/, https://www.n...",5,Actaea pachypoda,White Baneberry


In [18]:
final["String"] = [f"{row['Common Name']} ({row['Scientific Name']}): {row['URL']}" for _,row in final.iterrows()]

In [19]:
final.head()

,USDA Symbol,URL,COUNT,Scientific Name,Common Name,String
0,ABBA,"https://www.musserforests.com/, http://www.per...",4,Abies balsamea,Balsam Fir,Balsam Fir (Abies balsamea): https://www.musse...
1,ACAM,https://edgeofthewoodsnursery.com/,1,Acorus americanus,Sweetflag,Sweetflag (Acorus americanus): https://edgeoft...
2,ACMI2,https://www.ginosnursery.com/,1,Achillea millefolium,Common Yarrow,Common Yarrow (Achillea millefolium): https://...
3,ACNE2,"https://archewild.com/, https://www.gonativetr...",3,Acer negundo,Boxelder,Boxelder (Acer negundo): https://archewild.com...
4,ACPA,"https://www.collinsnursery.com/, https://www.n...",5,Actaea pachypoda,White Baneberry,White Baneberry (Actaea pachypoda): https://ww...


In [20]:
final.to_excel("LOCAL_AGG.xlsx")